In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
from tensorflow.python.ops import rnn, rnn_cell
import random
import string

# Hyperparameters

In [15]:
input_size = 32769
hidden_layer = 5

normal = 18

sampling_rate = 200
striding = 100
train_steps = 2000

# Onset-time data

In [3]:
def seizure_time_parser(onset_times,patient_number,data_set_number):
    temp = str(onset_times[patient_number-1][data_set_number])
    curr_onset_time = []
    while temp.find('/')!=-1:
        curr_onset_time.append(int(temp[0:temp.find('/')]))
        temp=temp[temp.find('/')+1:]
    curr_onset_time.append(int(float(temp)))
    curr_onset_time=np.array(curr_onset_time)
    return curr_onset_time

In [4]:
seizure_file = open('/home/maestroj/medical_analysis/project2/seizure_times.csv','r',newline='')
reader = csv.reader(seizure_file,delimiter=',')
onset_times_temp=[]

for row in reader:
    onset_times_temp.append(row)
    
onset_times = []
total_patient = len(onset_times_temp)
event_number = np.zeros(total_patient,dtype=np.int32)
#number of events of (i+1) patient = event_number[i]

for i in range(total_patient):
    event_number[i] = len(onset_times_temp[i])-1

for p in range(total_patient):
    temp = []
    for d in range(event_number[p]):
        temp.append(seizure_time_parser(onset_times_temp,p+1,d+1))
    onset_times.append(temp)

In [5]:
print("The number of datasets of patients")
print(event_number)
print("1st patient's seizure time for each data_set")
print(onset_times[0][1])
print("9th patient's seizure time for each data_set")
print(onset_times[8])

The number of datasets of patients
[ 7  7  6  3 10  1  7 13 16  8  8  5  3  6 31  8  2  4]
1st patient's seizure time for each data_set
[781]
9th patient's seizure time for each data_set
[array([200]), array([312]), array([624]), array([762]), array([752]), array([772]), array([884]), array([502]), array([ 94, 258]), array([836]), array([650]), array([636]), array([576]), array([656]), array([638]), array([706])]


In [6]:
def stft_load(patient_number, dataset_number):
    input_folder_path = '/home/maestroj/medical_analysis/stft_data/'
    input_data_name = '{}_{}.csv'.format(patient_number,dataset_number)
    input_data_path = os.path.join(input_folder_path,input_data_name)
    input_file = open(input_data_path,'r',newline='')
    reader = csv.reader(input_file, delimiter=',')
    print("{} is loaded".format(input_data_name))
    temp = []
    for row in reader:
        temp.append(row)
    try:
        temp = np.array(temp, dtype=np.float32)
    except: 
        for i in range(len(temp)):
            for j in range(len(temp[0])):
                temp[i][j]=abs(complex(temp[i][j].replace('i','j')))
        temp = np.array(temp, dtype=np.float32)
    
    temp = np.transpose(temp)
    
    return temp

In [7]:
def stft_normal(dataset_number):
    input_folder_path = '/home/maestroj/medical_analysis/stft_normal'
    input_data_name = '{}.csv'.format(dataset_number)
    input_data_path = os.path.join(input_folder_path,input_data_name)
    input_file = open(input_data_path,'r',newline='')
    reader = csv.reader(input_file, delimiter=',')
    print("{} is loaded".format(input_data_name))
    temp = []
    for row in reader:
        temp.append(row)
    try:
        temp = np.array(temp, dtype=np.float32)
    except: 
        for i in range(len(temp)):
            for j in range(len(temp[0])):
                temp[i][j]=abs(complex(temp[i][j].replace('i','j')))
        temp = np.array(temp, dtype=np.float32)
        
    temp = np.transpose(temp)
    
    return temp

In [8]:
def time_axis_maker(array, sec_per_cell,init = 0):
    final = init + sec_per_cell*(len(array)-1)
    return np.linspace(init,final,len(array))

In [9]:
temp = stft_load(8,1)
print(temp.shape)

8_1.csv is loaded
(499, 32769)


# Training_set

In [11]:
input_data = []
for i in range(total_patient):
    for j in range(event_number[i]):
        try:
            temp = stft_load(i+1,j+1)
            print(temp.shape)
            for k in range(len(temp)):
                input_data.append(temp[k])
            
            
        except FileNotFoundError:
            continue;

1_1.csv is loaded
(499, 32769)
1_2.csv is loaded
(499, 32769)
1_3.csv is loaded
(499, 32769)
1_4.csv is loaded
(499, 32769)
1_5.csv is loaded
(499, 32769)
1_6.csv is loaded
(499, 32769)
1_7.csv is loaded
(499, 32769)
2_1.csv is loaded
(499, 32769)
2_2.csv is loaded
(499, 32769)
2_3.csv is loaded
(499, 32769)
2_4.csv is loaded
(499, 32769)
2_5.csv is loaded
(499, 32769)
2_6.csv is loaded
(499, 32769)
2_7.csv is loaded
(499, 32769)
3_1.csv is loaded
(499, 32769)
3_5.csv is loaded
(499, 32769)
3_6.csv is loaded
(499, 32769)
4_1.csv is loaded
(499, 32769)
4_2.csv is loaded
(499, 32769)
4_3.csv is loaded
(499, 32769)
5_2.csv is loaded
(499, 32769)
5_3.csv is loaded
(499, 32769)
5_4.csv is loaded
(499, 32769)
6_1.csv is loaded
(499, 32769)
7_1.csv is loaded
(499, 32769)
7_2.csv is loaded
(499, 32769)
7_3.csv is loaded
(499, 32769)
7_4.csv is loaded
(499, 32769)
7_5.csv is loaded
(499, 32769)
7_6.csv is loaded
(499, 32769)
7_7.csv is loaded
(499, 32769)
8_1.csv is loaded
(499, 32769)
8_2.csv 

NameError: name 'sstft_normal' is not defined

In [12]:
for i in range(normal):
    try:
        temp = stft_normal(i)
        print(temp.shape)
        for k in range(len(temp)):
            input_data.append(temp[k])
    except FileNotFoundError:
        continue;

input_data = np.array(input_data)
print(input_data.shape)

1.csv is loaded
(499, 32769)
2.csv is loaded
(499, 32769)
3.csv is loaded
(499, 32769)
4.csv is loaded
(499, 32769)
5.csv is loaded
(499, 32769)
6.csv is loaded
(499, 32769)
7.csv is loaded
(499, 32769)
8.csv is loaded
(499, 32769)
9.csv is loaded
(499, 32769)
10.csv is loaded
(499, 32769)
11.csv is loaded
(499, 32769)
12.csv is loaded
(499, 32769)
13.csv is loaded
(499, 32769)
14.csv is loaded
(499, 32769)
15.csv is loaded
(499, 32769)
16.csv is loaded
(499, 32769)
17.csv is loaded
(499, 32769)
(61876, 32769)


# Neural network

In [17]:
X = tf.placeholder(tf.float32,[None,input_size])

w1 = tf.Variable(tf.random_uniform([input_size,hidden_layer], -0.01, 0.01))
b1 = tf.Variable(tf.random_uniform([1, hidden_layer], -0.01, 0.01))

w2 = tf.Variable(tf.random_uniform([hidden_layer,input_size], -0.01, 0.01))
b2 = tf.Variable(tf.random_uniform([1, input_size], -0.01, 0.01))

H = tf.matmul(X, w1) + b1
hypothesis = tf.matmul(H, w2) + b2

cost = tf.reduce_mean(tf.square(X-hypothesis))
optimizer = tf.train.AdamOptimizer(1e-3)
train = optimizer.minimize(cost)

In [20]:
sess = tf.Session()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

batch_number= 100
batch_size = int(len(input_data)/batch_number)
for i in range(batch_number):
    for step in range(train_steps+1):
        sess.run(train, feed_dict={X: input_data[i*batch_size:(i+1)*batch_size]})
        if step%100==0:
            print("{}th run cost : {} ".format(step,
                                        sess.run(cost,feed_dict={X: input_data[i*batch_size:(i+1)*batch_size]})))

0th run cost : 124641.3203125 
100th run cost : 15637.736328125 
200th run cost : 12545.3115234375 
300th run cost : 9894.990234375 
400th run cost : 9864.630859375 
500th run cost : 9861.4580078125 
600th run cost : 9860.7890625 
700th run cost : 9860.634765625 
800th run cost : 9865.08203125 
900th run cost : 9860.5166015625 


KeyboardInterrupt: 

In [13]:
for i in range(total_patient):
    for j in range(event_number[i]):
        try:
            temp = stft_load(i+1,j+1)
            temp_output = sess.run(H, feed_dict = {X : temp})
            temp_output = np.transpose(temp_output)
            
            output_file_path = '/home/maestroj/medical_analysis/stft_data_reduced'
            output_file_name = '{}_{}.csv'.format(i+1, j+1)
            output_file_name = os.path.join(output_file_path,out_file_name)
            
            print("Reduced data is stored in {}".format(output_file_name))
            
            csv_file = open(output_file_name1, "w")
            cw = csv.writer(csv_file , delimiter=',')
            size = train_length
            
        except FileNotFoundError:
            continue;
            
for i in range(normal):
    try:
        temp = stft_normal(i)
        print(temp.shape)
        for k in range(len(temp)):
            input_data.append(temp[k])
    except FileNotFoundError:
        continue;

input_data = np.array(input_data)
print(input_data.shape)

1_1.csv is loaded


NameError: name 'sess' is not defined